In [ ]:
import pandas as pd
import datetime
import json
from datetime import datetime
from datetime import timedelta

# Define the important variables here

target = {
    #"fips": 6071,
    "target_county_name": "Calaveras",
    "date_column_name": "DATE",
    "datetime_format": "%Y%m%d",
    "target_data_file_name": "",
    "target_data_field_name": "prcp"
}
target['target_data_file_name'] = f"Datasets/merged_tp_precip_wind_fmc_{target['target_county_name']}.csv"

# Use the adjacent county of the target county to find referral data like aws or fmc
# Use the following link to find the adjacent county
# https://gis.data.ca.gov/datasets/8713ced9b78a4abb97dc130a691a8695/explore?location=39.765076%2C-121.456785%2C8.00
referral = {
    "referral_fips": 6051,
    "referral_country_name": "Tuolumne",
    "date_column_name": "date",
    "datetime_format": "%Y%m%d",
    "referral_data_file_name": "Datasets/tp_zipcode_county_CA.csv",
    "referral_data_field_name": "prcp"
}


# Extract the whole volume 
def filter_dataframe_by_value(df, column_name, value_to_find):
    filtered_rows = df[df[column_name] == value_to_find]
    return filtered_rows

# Copy the matched column from the referral dataset to the target dataset
# The match key is the date. Note the date column might have different names
# The datetime must be close to each other between two datasets
def merge_dataframes_on_match(target_df, referral_df, 
                            target_data_field_name, referral_data_field_name):
    #referral_df.head()
    for index1, row1 in target_df.iterrows():
        #print(index1)
        # Convert to a datetime object
        row1_datetime = datetime.strptime(str(row1[target['date_column_name']]),  target['datetime_format'])
        # Iniatize the second datetime
        row2_datetime = row1_datetime; 
        data_to_fill = -1
        data_to_fill_datetime = row2_datetime
        data_to_fill_delta_days = 0
        data_to_fill_row = {}
        for index2, row2 in referral_df.iterrows():
            #print(row2)

            row2_datetime = datetime.strptime(str(row2[referral['date_column_name']]), referral['datetime_format'])
            delta_days = (row1_datetime - row2_datetime).days
            if (delta_days < 0):
                delta_days = -delta_days
                
            #print(f"Found a row with diff days {delta_days} between {row1_datetime} and {row2_datetime}")
            
            if (delta_days < 100):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
            if (delta_days < 50):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
            if (delta_days < 25):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
            if (delta_days < 10):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
            if (delta_days < 5):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
            if (delta_days < 3):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
            if (delta_days < 1):
                data_to_fill = row2[referral_data_field_name]
                data_to_fill_datetime = row2_datetime
                data_to_fill_delta_days = delta_days
                data_to_fill_row = row2
                break   

        if (data_to_fill != -1):
            print(f"Add data to the original {row1_datetime} <- {data_to_fill_datetime} - {data_to_fill_delta_days} days: {data_to_fill}")
            target_df.at[index1, target_data_field_name] = data_to_fill
            target_df.at[index1, 'tmax'] = data_to_fill_row['tmax']
            target_df.at[index1, 'tmin'] = data_to_fill_row['tmin']
            target_df.at[index1, 'tavg'] = data_to_fill_row['tavg']

#
# Main program starts here
#
def merge_data_referal_to_target(target, referral): 

    # Load the target dataset
    target_dataset = pd.read_csv(target['target_data_file_name'])
    print(f"The dataset {target['target_data_file_name']} contains (row, column) = ")
    print(target_dataset.shape)

    # Initial the referral dataset from a csv
    referral_dataset = pd.read_csv(referral['referral_data_file_name'])

    # Check if the county exists in the referral dataset
    referral_count = referral_dataset['county'].str.contains(referral['referral_country_name']).sum()
    print(f"The referral county {referral['referral_country_name']} has {referral_count} rows of data in the {referral['referral_data_file_name']}")  
    # If no data is found, exit the program
    if ( int(referral_count) <= 0):
        return 
        # raise SystemExit("The county doesn't exist in the referral dataset. Exits the program. ")
    
    # Create a new data frame containing only rows with the specific value
    referral_dataset_fips_only = filter_dataframe_by_value(referral_dataset, 'fips', referral['referral_fips'])
    print( f"referral_dataset with fips {referral['referral_fips']} = {referral_dataset_fips_only.shape}")

                
    # Add the new data from the referral dataset to the target dataset
    merge_dataframes_on_match(target_dataset, referral_dataset_fips_only.sort_values("date"), 
                            target['target_data_field_name'], referral['referral_data_field_name'])

    # Save the data to its original file
    target_dataset.to_csv(target['target_data_file_name'])
    print(f"Saved to {target['target_data_file_name']}")

#
# Main program
#
merge_data_referal_to_target(target, referral)
